# **Word Prediction with Levenshtein Distance**

The simplest model that can be extremely useful for word prediction is using Levenshtein Distance (or Edit Distance) between two strings. This is a number, calculated by counting how many of the following operations are necessary to turn one string into the other:

* Remove a character  
* Add a character  
* Change a character

Therefore, we can create a prediction model that, given a vocabulary of words, returns to the user the top 3 words with the smallest edit distance from the inputted sequence.

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pdfplumber
import Levenshtein
import pandas as pd

To add some informal and conversational words to the vocab, we are scanning movie scripts! For that, we need to read the pdf's included in the [data](/data) folder:

In [4]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = []
        for i, page in enumerate(pdf.pages):
            # Skip the first page (it's the title page)
            if i == 0:
                continue
            
            # Extract text from the page
            page_text = page.extract_text()
            
            # Ignore the last line of each page (it's the page number)
            lines = page_text.split("\n")
            if lines:
                page_text = lines[:-1]
            
            # Add the text to the list
            text += list(map(lambda x: x.strip().lower(), page_text))

    return text

Now, we are going to create our vocabulary by concatenating all the words we find on our 3 sources:

1. Movie Scripts  
2. 5k Word Frequency List  
3. Compilation of WikiNews

In [6]:
gu1 = extract_text_from_pdf('../data/grown-ups.pdf')
gu2 = extract_text_from_pdf('../data/grown-ups-2.pdf')

df_words = pd.read_excel('../data/wordFrequency.xlsx', sheet_name='4 forms (219k)')
words = df_words['word'].values

with open('../data/en_wikinews.txt', 'r', encoding='utf-8') as f:
    en_wikinews = f.readlines()

In [7]:
len(gu1), len(gu2), len(en_wikinews), len(words)

(1863, 2951, 167231, 5050)

In [8]:
gu1[:5]

['two-handed.',
 "- why, because i'm black?",
 "yes, that's what i thought",
 'ten seconds.',
 "we're going to win."]

In [9]:
en_wikinews[:5]

['in a statement today senior investigating officer detective superintendent mike veale named hugo quintas aged 23 of seymour court trowbridge former boyfriend of hayley richards as being wanted for her murder and stated that a european warrant had been issued for his arrest . richards aged 23 and pregnant at the time of her murder was found in her flat in carders corner trowbridge on the afternoon of saturday june 11 2005 with her throat slit . ds veale stated that it was known that quintas boarded a plane from bristol airport on saturday afternoon bound for portugal but that his present whereabouts are unknown .\n',
 'the first trial ended in a mistrial after one juror received a threatening letter . nancy salomon says of the second trial that the defense was counting on the jury not reaching a verdict they had used their challenges during the jury selection process to kick almost every potential juror who had a college degree or had any business experience or wall street experience 

In [10]:
words[:5]

array(['the', 'to', 'and', 'of', 'a'], dtype=object)

## Concatenating unique words

For this step, we are going to create a set of all the words on our vocabulary. Also, we are keeping track of the frequency of each one, to try to incorporate that into our model:

In [14]:
# Now it's time to create a vocabulary, let's use a flatmap to extract all the words from the text
def flatmap(text):
    return [word for line in text for word in line.split() if word.isalpha()]

def create_vocab(*texts):
    vocab = {}
    for text in texts:
        for word in text:
            vocab[word] = vocab.get(word, 0) + 1

    return vocab

In [15]:
vocab = create_vocab(flatmap(gu1), flatmap(gu2), flatmap(en_wikinews), [str(word) for word in words])

In [16]:
vocab, len(vocab)

({'because': 32679,
  'what': 44517,
  'i': 180829,
  'thought': 5762,
  'ten': 3373,
  'going': 16176,
  'to': 613720,
  'just': 32538,
  'do': 42171,
  'his': 57372,
  'foot': 747,
  'was': 147826,
  'on': 158799,
  'the': 1236935,
  'first': 26190,
  'want': 15909,
  'thank': 1433,
  'earnshaw': 1,
  'family': 7756,
  'for': 198706,
  'their': 61637,
  'summer': 1025,
  'and': 517291,
  'celebrate': 326,
  'our': 34830,
  'five': 5628,
  'they': 93088,
  'are': 111427,
  'we': 91202,
  'see': 19327,
  'here': 15519,
  'subject': 2802,
  'of': 542855,
  'children': 7673,
  'not': 117869,
  'talk': 22893,
  'about': 50321,
  'your': 20812,
  'have': 123197,
  'performed': 1059,
  'this': 113445,
  'you': 84695,
  'played': 2409,
  'game': 6840,
  'like': 33676,
  'always': 6841,
  'gave': 3280,
  'everything': 4143,
  'when': 40224,
  'final': 3574,
  'buzzer': 25,
  'went': 6386,
  'off': 11670,
  'were': 60416,
  'promise': 573,
  'me': 23796,
  'something': 12378,
  'play': 4436,
 

## **Top K Nearest Words**

Now, it's time to use the Edit Distance to get the top words.

In [20]:
def find_top_k_nearest_words(query_input, words_list, k): 
    distances = [] 
    for word in words_list: 
        distance = Levenshtein.distance(query_input, word) 
        distances.append((word, distance)) 

    # Sort the list by the distance
    sorted_distances = sorted(distances, key=lambda x: x[1]) 

    # Return the top k nearest words
    return [d[0] for d in sorted_distances[:k]] 
  
nearest_words = find_top_k_nearest_words("pytorch", vocab, 3)
print(nearest_words)

['porch', 'torch', 'touch']


What if we also try to introduce the frequency of the word into the calculation?

In [21]:
def find_top_k_nearest_words_frequency(query_input, words_list, k, max_freq): 
    distances = []
    for word in words_list: 
        distance = Levenshtein.distance(query_input, word) 
        distances.append((word, distance - words_list[word]/max_freq))  # Subtract the frequency of the word from the distance
                                                                        # to give more weight to more frequent words, that are gonna have smaller distances
    # Sort the list by the distance
    sorted_distances = sorted(distances, key=lambda x: x[1]) 

    # Return the top k nearest words
    return sorted_distances[:k]

In [26]:
def practice_predict(vocab):
    MAX_FREQ = max(vocab.values())
    
    while True:
        query_input = input("Enter a word: ")
        if query_input == "exit":
            break
        nearest_words = find_top_k_nearest_words_frequency(query_input, vocab, 3, MAX_FREQ)
        print(f"Nearest words to {query_input}: {nearest_words}")

In [27]:
practice_predict(vocab)

Nearest words to cal: [('cal', -4.446474552017689e-05), ('can', 0.965009479075295), ('call', 0.9959666433563606)]
Nearest words to pytorch: [('torch', 1.9999490676551315), ('porch', 1.9999692789030952), ('touch', 2.9992319725773786)]
Nearest words to freqncy: [('french', 1.9980201061494742), ('frequency', 1.9998536705647427), ('frenzy', 1.9999636197536652)]
Nearest words to alpha: [('alpha', -0.00011641678827100858), ('alpa', 0.9999854479014662), ('ralph', 1.999767166423458)]
Nearest words to alphabtws: [('alphabet', 2.999962811303747), ('alpha', 3.999883583211729), ('elephants', 3.999898135310263)]


Maybe this is also useful for autocorrecting!

# **V2 - 100k**

Let's also create a second version testing the 100k vocab:

In [34]:
with open('../data/wiki-100k.txt', 'r', encoding='utf-8') as f:
    words = f.readlines()
    words = list(map(lambda x: x.removesuffix("\n"), filter(lambda x: '#' not in x, words)))
    word2rank = {}

    def preprocess_text(text):
        # filter all capitalized words
        words = [word for word in text if word.islower()]

        # this function can be improved in the future!

        return words
    
    words = preprocess_text(words)

    # Create a dictionary with the word as the key and the rank as the value
    i = 1
    for word in words:
        if word not in word2rank:
            word2rank[word] = i
            i += 1

There is a disadvantage in this approach: we don't have the frequency of the word, but only his ranking on the list! This is going to change our calculations on the distance.

In [35]:
list(word2rank.items())[:5], len(words)

([('the', 1), ('of', 2), ('and', 3), ('to', 4), ('a', 5)], 65758)

Now, let's implement our new function to find the top k suggestions:

In [36]:
# K-nearest words but considering the ranking
def find_top_k_nearest_words_ranking(query_input, word2rank, k): 
    distances = []
    for word in word2rank: 
        distance = Levenshtein.distance(query_input, word) 
        distances.append((word, distance - 3 * (len(word2rank) - word2rank[word])/len(word2rank)))  # Subtract 3 times the "relative ranking" of the word from the distance
                                                                                                    # to give more weight to more frequent words, that are gonna have smaller distances
    sorted_distances = sorted(distances, key=lambda x: x[1])
    
    return sorted_distances[:k]

In [37]:
def practice_predict_ranking():
    while True:
        query_input = input("Enter a word: ")
        if query_input == "exit":
            break
        nearest_words = find_top_k_nearest_words_ranking(query_input, word2rank, 3)
        print(f"Nearest words to {query_input}: {nearest_words}")

In [38]:
practice_predict_ranking()

Nearest words to cal: [('can', -1.9954212113315712), ('call', -1.9783345121542641), ('col', -1.9630904962215685)]
Nearest words to pytorch: [('porch', -0.7118713472061944), ('torch', -0.5485426050701707), ('touch', 0.05745821390016026)]
Nearest words to freqncy: [('frenzy', -0.5347504001786842), ('frequency', -0.30078546699921827), ('freely', 0.06845847448162923)]
Nearest words to alpha: [('alpha', -1.278375460670811), ('alla', -0.7725682165059746), ('alma', -0.18447306704388922)]
Nearest words to alphabtws: [('alphabet', 0.6036183598257825), ('elephants', 1.4249897628708634), ('alphabets', 1.8225998585414882)]


The results are fairly similar for now! More inspection will be needed.